# Telemática

En este cuaderno se presenta una guía de configuración de algunos servicios en máquinas Ubuntu en la plataforma AWS.

Realizado por:

Ferney O. Amaya Fernández <BR>
ferney.amaya@gmail.com

#1.Regiones, Zonas de Disponibilidad y VPCs

Amazon cuente con regiones en todo el mundo, por ejemplo:
- us-east-1 (Norte de Virginia, EE. UU.) con 6 zonas de disponibilidad
- eu-west-1 (Irlanda)
- ap-southeast-1 (Singapur)

En cada región se tienen varias Zonas de Disponibilidad (*Availability Zones, AZ*). Una AZ es un grupo de centros de datos con alimentación, refrigeración y seguridad física independientes.

[Mayor información aquí.](https://aws.amazon.com/es/about-aws/global-infrastructure/)

En una VPC (*Virtual Private Cloud*) pueden lanzarse varios recursos de AWS en una red virtual lógica aislada que se asemeja a una red tradicional que conforma un centro de datos.

[Mayor información aquí.](https://docs.aws.amazon.com/es_es/vpc/latest/userguide/what-is-amazon-vpc.html)

#2 Paso a paso para crear una VPC manual

La VPC y sus componentes pueden crearse en forma automática y en forma manual. Aquí se presenta una breve guía para crearlas en forma manual.

Se creará una VPC con 1 subred pública y 1 subred privada. Se empleará el bloque CIDR IPv4 10.2.0.0/16 para crear subredes 10.2.0.0/18.

##2.1 **Paso 1:** Crear la VPC

La configuración a seleccionar es la siguiente:
- Resources to create: VPC only
- IPv4 CIDR manual input:
- IPv6 CIDR block: Amazon-provided IPv6 CIDR block
- Tenancy: Default (sin HW dedicado, más barato)

##2.2 **Paso 2:** Crear las subredes

Las subredes públicas y privadas se crean igual. La diferencia estará más adelante en la tabla de enrutamiento al conectar o no con la puerta de enlace.

- Seleccione la VPC
- Identificar y planear el direccionamiento IPv4 e IPv6
- Asigne un nombre que indique si será pública o privada
- Seleccione AZ
- Seleccione IPv4 e IPv6 subnet CIDR block

##2.3 **Paso 3:** Crear el GW (Internet gateways)

El GW (*Gateway*) o puerta de enlace:
- Crear el GW
- Editarlo para conectarlo con la VPC:
 * En Actions selecionar Atach to VPC
 * Seleccionar la VPC

##2.4 **Paso 4:** Crear las Tablas de Enrutamiento

Es suficiente una tabla para las subredes públicas con el mismo GW y otra para las privadas.
- Seleccione la VPC y cree la tabla de enrutamiento
- Edite la tabla de enrutamiento para asociarla a la subred:
  - Seleccione la pestaña Subnet Associations
  - En la sección Subnet without explicit associations editar para selecionar la subred



##2.5 **Paso 5:** Ajustar las rutas en las Tablas de Enrutamiento

La tabla de enrutamiento para la subred pública debe ajustarse para que quede de la siguiente forma (el siguiente es un ejemplo, los datos deben ajustarse dependiendo de las direcciones IP):

|Destino|Destino|
|--|--|
|::/0|igw-0d7ec8f0c00a668ba|
|2600:1f10:4d0a:5700::/56|local|
|0.0.0.0/0|igw-0d7ec8f0c00a668ba|
|10.2.0.0/16|local|






#3.Enlace SSH entre cliente y servidor

En esta sección se creará el usuario *tele* en el Servidor y se habilitará para realizar una conexión remota ssh desde el Cliente.

Necesitará abrir dos consolas. Puede ser util cambiar el prompt para que refleje el nombre Cliente o Servidor a la vez que se visualice la IP. Por ejemplo, si el prompt actual es:
```
ip-172-32-42-156
```
Debe cambiarse el nombre del host en el archivo:
```
/etc/hostname
```
colocando para el servidor el nombre:
```
Server-172-32-42-156
```
Para que el cambio se refleje en el prompt, deben ejecutarse los comandos:
```
sudo hostname Server-172-32-42-156
exec bash
```

##3.1 Revisión e instalación

Verificar cliente:
```
ssh –V
```

Verificar servidor:
```
sudo systemctl status ssh
```

##3.2 En el servidor crear usuario

Crear el usuario *tele* con contraseña *tele*.

##3.3 En el servidor ajustar la configuración

En el archivo de configuración del servidor:

```
/etc/ssh/sshd_config
```

Quitarle el comentario y asignar el siguiente valor a las opciones:
- Port 22
- PasswordAuthentication yes
- PubkeyAuthentication yes
- KbdInteractiveAuthentication yes

La opción *KbdInteractiveAuthentication* debe estar en *yes* para permitir interacción remota desde teclado.
Para la autenticación en el servidor ssh hay dos opciones:
- Si *PubkeyAuthentication* está en *yes* la autenticación será por clave pública.
- Si *PubkeyAuthentication* está en *no* y *PasswordAuthentication* en *yes* la autenticación será por password.

##3.4 En el cliente generar y verificar el par de llaves

Para generar el par de llaves se sugiere:
```
ssh-keygen -t rsa -b 4096
```
Las llaves se crean en las siguientes ubicaciones:
- ~/.ssh/id_rsa	: Clave privada, no compartir.
- ~/.ssh/id_rsa.pub	: Clave pública, puede compartirse
Observe las llaves.

##3.5 Desde el cliente copiar la clave pública al servidor

El comando para enviar desde el cliente la clave pública al servidor es el siguiente:
```
 ssh-copy-id username@IP_servidor
```

Si el comando anterior no funciona, la clave debe copiarse manualmente. Siga los siguientes pasos:

- Paso 1: En el Servidor, ingrese a la sesión del usuario tele.
- Paso 2: En el directorio *home* crear el directorio *.ssh* con los siguientes privilegios:
```
 drwx------      .ssh
```
- Paso 3: En el directorio *.ssh* crear el archivo *authorized_keys* con los siguientes privilegios:
```
 -rw-------      authorized_keys
```
- Paso 4: Copie manualmente la llave pública creada en el Cliente, al archivo *authorized_keys* en el servidor.


#4.UFW
La configuración del servicio UFW (*Uncomplicated FireWall*) es la siguiente:

##4.1. Configurar las políticas por defecto
```
sudo ufw default deny incoming​
sudo ufw default allow outgoing​
```

##4.2. Habilitar servicios

El mapeo de servicios a números de puertos se encuentra en el archivo:
```
/etc/services
```
Para agregar un servicio puede hacerse por el nombre de servicio o por número de puerto:
```
sudo ufw allow servicio​
sudo ufw allow numero_puerto​
```

##4.3. Controlar UFW​
Para controlar el servicio UFW se emplean los siguientes comandos:
```
sudo ufw enable​
sudo ufw status verbose​
sudo ufw disable​
sudo ufw status numbered
```

#5.DHCP

El archivo de configuración del servicio:
```
/etc/dhcp/dhcpd.conf
```
Un ejemplo de configuración es la siguiente:
```
default-lease-time 600;
max-lease-time 7200;
subnet 192.168.1.0 netmask 255.255.255.0 {
  range 192.168.1.150 192.168.1.200;
  option routers 192.168.1.254;
  option domain-name-servers 192.168.1.1;
  option domain-name "mydomain.example";
```


#6.Configuración básica en una máquina

1. Ajustar archivo hosts:
```
  /etc/hosts
```
Agrega debajo de la línea 127.0.0.1   localhost, una línea como esta (verifica la IP):
```
  127.0.1.1   ip-10-100-1-10
```

2. Ajustar el archivo resolv.conf:
```
  /etc/resolv.conf
```
No remover la línea 127.0.0.53, coméntela. Colocar el servidor de nombre, que para este ejemplo es:
```
  nameserver 10.100.1.10
```
Agregue el dominio:
```
  domain telematica.com
```
Debe tenerse en cuenta que el sistema operativo actualiza frecuentemente este archivo.

#7.DNS

Se realizará un ejemplo de configuración con la siguiente información:
- Nombre de dominio: telematica.com
- Dirección de red: 10.100.1.0/24

Con esta información, los datos para el servicio son:
-	zona directa: **telematica.com**
- zona inversa: **1.100.10.in-addr.arpa**
Se seleccionan los nombres:
- Nombre de archivo directo: **a_directo**
- Nombre de archivo inverso: **a_inverso**

**Paso 1:** Instalar bind9 en el servidor:
```
  sudo apt install bind9
```

**Paso 2:** Crear y verificar las zonas:
1. Abra el siguiente archivo:
```
  /etc/bind/named.conf.local
```

2. Incluya en el archivo el siguiente contenido:
```
  zone "telematica.com" {
    type master;
    file "a_directo"
  ; };

  zone "1.100.10.in-addr.arpa" {
    type master;
    file "a_inverso"
  ; };
```
3. Verifique la edición del archivo
```
	named-checkconf
```

**Paso 3:** Crear y verificar el archivo del mapa directo:
1. Cree el archivo del mapa directo:
```
  /var/cache/bind/a_directo
```
2. Incluya en el archivo el siguiente contenido:
```
$ORIGIN telematica.com.
$TTL 86400 ; 1 dia
@ IN SOA servidor postmaster (
120219 ; serie
6H ; refrescamiento (6 horas)
1H ; reintentos (1 hora)
2W ; expira (2 semanas)
3H ; mínimo cache (3 horas)
)
      IN NS server
server IN A 10.100.1.10
client IN A 10.100.1.20
```
3. Verifique la edición del archivo
La sintaxis es:
```  
  named-checkzone zona_directa ruta/a_directo
```

**Paso 4:** Crear y verificar el archivo del mapa inverso:
1. Cree el archivo del mapa directo:
```
  /var/cache/bind/a_inverso
```
2. Incluya en el archivo el siguiente contenido:
```
$ORIGIN 1.100.10.in-addr.arpa.
$TTL 86400 ; 1 dia
@ IN SOA servidor postmaster (
120219 ; serie
6H ; refresco (6 horas)
1H ; reintentos (1 hora)
2W ; expira (2 semanas)
3H ; mínimo (3 horas)
)
      IN NS server.telematica.com.
10 IN PTR server.telematica.com.
20 IN PTR client.telematica.com.
```
3. Verifique la edición del archivo
```  
  named-checkzone zona_inversa ruta/a_inverso
```

**Paso 5 (opcionial):** Permitir la resolución externa

Para resolver nombres de dominios externos debe editarse el archivo:
```
  /etc/bind/named.conf.options
```
Verificar:
```
options {
    directory "/var/cache/bind";

    recursion yes;
    allow-recursion { any; };   #resolución externa

    forwarders {
        8.8.8.8;   # DNS de Google
        8.8.4.4;   # DNS de Google
    };
```

**Paso 6:** Reiniciar el servidor:
```
 	sudo systemctl restart bind9
```

Para observar el estado del servidor:
```
 	sudo systemctl status bind9
```


#8.Correo electrónico

Se realizará un ejemplo de configuración teniendo en cuenta lo siguiente:
- Se asume que en la máquina está instalado un servidor DNS bind9
- Nombre de dominio: telematica.com

Actualice el sistema operativo



##8.1 Configurar postfix y dovecot

**Paso 1:** Actualizar el sistema operativo

**Paso 2:** Instalar Postfix (servidor SMTP)
```
  sudo apt install postfix
```
Durante la instalación:
- Selecciona "Internet Site"
- Ingresar el nombre de dominio

**Paso 3:** Configurar Postfix

Edita el archivo de configuración principal:
```
  /etc/postfix/main.cf
```
Asegúrate de que contenga estas líneas:
```
  myhostname = mail.midominio.com
  mydomain = midominio.com
  myorigin = $mydomain
  inet_interfaces = all
  mydestination = $myhostname, localhost.$mydomain, localhost, $mydomain
  mynetworks = 127.0.0.0/8, [IP de tu red local]/24
  relayhost =
  home_mailbox = Maildir/
```

**Paso 4:** Instalar Dovecot (servidor IMAP/POP3)
```
  sudo apt install dovecot-imapd dovecot-pop3d
```

**Paso 5:** Configurar Dovecot

Edita el archivo de configuración:
```
  /etc/dovecot/conf.d/10-mail.conf
```
Asegúrate de que contenga:
```
  mail_location = maildir:~/Maildir
```

**Paso 6:** Reiniciar servicios

```
  sudo systemctl restart postfix
  sudo systemctl restart dovecot
```

**Paso 7:** Ajustar el servicio DNS

Ajustar archivo DNS directo para adicionar el servidor de correo. Al arvhivo del mapa directo debe adicionarse la línea del servidor de correo:

```
...
      IN NS server
      IN MX 10 server
...
```

Recuerde que cada cambio hecho al DNS implica que se debe chequear el archivo con el comando *named-checkzone* y luego debe reiniciar el servicio bind9.

##8.2 Verificar con Telnet

Establecer una conexión Telnet a través del puerto 25 para generar un correo electrónico desde la cuenta ubuntu@telematica.com a la cuenta ubuntu@telematica.com empleando SNMP:
```
  telnet localhost 25
```
Se presenta un ejemplo de como generar el correo:

```
  $ telnet localhost 25
  Trying 127.0.0.1...
  Connected to localhost.
  Escape character is '^]'.
  220 server.telematica.com ESMTP Postfix (Ubuntu)
  ehlo telematica.com
  250-server.telematica.com
  250-PIPELINING
  250-SIZE 10240000
  250-VRFY
  250-ETRN
  250-STARTTLS
  250-ENHANCEDSTATUSCODES
  250-8BITMIME
  250-DSN
  250-SMTPUTF8
  250 CHUNKING
  mail from:ubuntu@telematica.com
  250 2.1.0 Ok
  rcpt to:ubuntu@telematica.com   
  250 2.1.5 Ok
  data
  354 End data with <CR><LF>.<CR><LF>
  Este es un mensaje de prueba
  .
  250 2.0.0 Ok: queued as 697F648940
  quit
  221 2.0.0 Bye
  Connection closed by foreign host.
```

Luego de enviar el correo, puede verificarse el correo electrónico recibido en la cuenta ubuntu puede verse en el archivo:
```
  ~/Maildir/new/
```



##8.3 Instalar Mailutils

Al intalar mailutils se instala Postfix:
```
  sudo DEBIAN_PRIORITY=low apt install mailutils
```
Las opciones de instalación que hay que ajustar en Postfix son:
- General mail configuration type: Internet Site
- System mail name: telematica.com
- Recipients for root and postmaster mail: root
- Force synchronous update on mail queue: YES
- Mailbox size limit: 0
- Local address extension character: +
- Internet protocols to use: all

En caso de no mostrar las posibilidades de configuración debe ejecutarse el comando:
```
   sudo dpkg-reconfigure postfix
```

**Ajustar postfix y mailutils**

Configuraremos el buzón con el formato Maildir que creará una estructura de directorio con ese nombre dentro del directorio personal del usuario:
```
  sudo postconf -e 'home_mailbox= Maildir/'
```
Configurar la variable de entorno MAIL para que el cliente pueda encontrar su correo:
```
  echo 'export MAIL=~/Maildir' | sudo tee -a /etc/bash.bashrc | sudo tee -a /etc/profile.d/mail.sh
```
Para leer la variable en su sesión actual:
```
  source /etc/profile.d/mail.sh
```

##8.4 Instalar cliente de correo electrónico mutt

**Paso 1:** Instalar mutt
```
  sudo apt install mutt
```

**Paso 2:** Configurar mutt

Crear el archivo de configuración:
```
  mkdir -p ~/.mutt
  nano ~/.mutt/muttrc
```

**Paso 3:** Agrega esta configuración

Al archivo anterior agregar (hacer las ediciones necesarias):
```
set from = "usuario1@midominio.com"
set realname = "Tu Nombre"
set smtp_url = "smtp://usuario1@[IP-del-servidor]:25/"
set smtp_pass = "contraseña"
set imap_user = "usuario1@midominio.com"
set imap_pass = "contraseña"
set folder = "imap://[IP-del-servidor]:143/"
set spoolfile = "+INBOX"
set header_cache = ~/.mutt/cache/headers
set message_cachedir = ~/.mutt/cache/bodies
```

**Paso 4:** Deshabilitar TLS/SSL

Si estás en una red local confiable y no necesitas cifrado, puedes deshabilitar TLS. Este procedimiento no es recomendado para producción. Esto se hace modificando la configuración de Mutt (~/.mutt/muttrc) agregando:

```
set ssl_starttls=no
set ssl_force_tls=no
set ssl_verify_dates=no
set ssl_verify_host=no
set certificate_file=~/.mutt/certificates
```


##8.5 Filtro de correo electrónico

El usuario puede instalar una aplicación como **procmail** para filtrar correo por usuario.

##8.6 Puertos

Los puertos a tener en cuenta son
- SMTP - Simple Mail Transfer Protocol: Puerto 25
- POP3 - Post Office Protocol 3: Puerto 110
- IMAP - Internet Message Access Protocol: Puerto 143

#9.Terraform y AWS CLI

##9.1 Instalar Terraform

Verificar si está instalado:
```
terraform -help
```

Para verificar el procesador:
```
uname -a
```

Si no está instalado, realizar la instalación:

https://developer.hashicorp.com/terraform/tutorials/aws-get-started/install-cli

##9.2 Instalar AWS CLI

Verificar si está instalado:
```
aws --version
```

Si no está instalado, realizar la instalación:

https://docs.aws.amazon.com/cli/latest/userguide/getting-started-install.html#cliv2-linux-install

##9.3 Credenciales AWS

Este procedimiento debe realizarse cada vez que se conecte a la cuenta de AWS, ya que durante cada conexión las credenciales cambian.

**Paso 1:** Inicie el laboratorio (*StartLab*)

**Paso 2:** Copiar las credenciales
- Hacer click en: *i AWS Details*, donde aparecen las instrucciones.
- Copiar las credenciales en el archivo:
```
 ~/.aws/credentials
```
Si el archivo no existe debe crearse.

**Paso 3:** Revise la región en: *iAWS Details*. Por ejemplo: us-east-1.

**Paso 4:** Complete la configuración con el comando:
```
aws configure
```
En el diálogo que aparece, deje todos campos como están (de ENTER) excepto en Region. En el campo región ingrese la región observada en el punto anterior.

**Paso 5:** Prueba de acceso a su cuenta:

Ejecute el comando:
```
aws ec2 describe-instances
```

debe observar las características de las EC2 que tiene creadas en su cuenta.


https://developer.hashicorp.com/terraform/tutorials/aws-get-started

##9.4 Lanzar una instancia EC2

**Paso 1:** Crear el archivo main.tf
Se sugiere crear un nuevo directoriio por cada configuración o archivo *main.tf*.

En este archivo adicione:
```
terraform {
  required_providers {
    aws = {
      source  = "hashicorp/aws"
      version = "~> 4.16"
    }
  }
  required_version = ">= 1.2.0"
}

provider "aws" {
  region = "us-east-1"
}

resource "aws_instance" "app_server" {
  ami           = "ami-07caf09b362be10b8"
  instance_type = "t2.micro"

  tags = {
    Name = "EC2_Terraform"
  }
}
```

Ajustes:
- Ajustar la región
- Ajustar el AMI ID, puede obtenerse de una EC2 ya creada o durante el proceso de creación.

**Paso 2:** Comandos Terraform:
1. Inicializar el directorio
```
terraform init
```

2.	Formatear y validar la configuración
```
terraform fmt
```

3.	Valida tu configuración:
```
terraform validate
```

4.	Crear infraestructura
```
terraform apply
```

5.	Inspeccionar el estado
```
terraform show
```

Para destruir el componente:
```
terraform destroy
```

https://developer.hashicorp.com/terraform/tutorials/aws-get-started/aws-build

#10.Flask

El siguiente archivo *server.py* renderiza una página html que se despliega en el puerto 8090.

```
from flask import Flask, render_template
app = Flask(__name__)

@app.route('/')
def index():
    return 'Bienvenido al curso de Telemática'

@app.route('/render')
def render():
    return render_template('hello.html')

if __name__ == "__main__":
    app.run (host="0.0.0.0", port=8090)
```

La siguiente es la estructura de archivos:

```
├── server.py
├── templates
    └── hello.html
```


#11.Scripting

- Shell: programa intermediario entre el usuario y el sistema operativo. Permite dar órdenes al sistema usando comandos. Interpreta los comandos que se escriben. Hay varios tipos de shell: sh, bash, zsh, csh, etc.

- Un script (o script shell) es un archivo de texto que contiene comandos de shell que se ejecutan de forma secuencial.

- Bash (Bourne Again SHell) tipo de intérprete de comandos o shell, muy común en Linux.

**Paso 1**

Probar lo siguiente:
- Verificar si el shell bash está instalado:
```
bash --version
```
- Identificar que shell se está usando:
```
echo $SHELL
```

**Paso 2**

Qué diferencia hy entre los siguientes dos comandos:

```
echo "TEXTO" > nombre_archivo
echo "TEXTO" >> nombre_archivo
```

**Paso 3**

Crea un script que use el shell bash para hacer lo siguiente en el home:
- Cree el directorio *dir*
- Cree el archivo *texto* dentro del directorio.
- Pregunte el nombre al usuario.
- Inserte en el archivo el texto "El nombre del usuario es:" y luego inserte el nombre.
- Muestre en pantalla el contenido del archivo.

Ayudas:
- La primera línea del script debe contener:
```
#!/bin/bash
```
- Para generar un mensaje en pantalla:
```
echo "Mensaje"
```
- Permitir que el usuario asigne valor a una variable:
```
read nombre_variable
```
- Presentar una variable en pantalla:
```
echo $nombre_variable
```
- Ejecutar un script desde la línea de comandos sin haberlo declarado ejecutable:
```
bash nombre_archivo
```
- Ejecutar un script desde la línea de comandos, luego de declarado ejecutable:
```
./archivo
```






#12.Docker

##12.1 Instalación

Pruebe si Docker está instalado:

```
docker --version
```

Si no está instalado, instálelo. Se recomienda el siguiente enlace, seleccionando la opción *Install using the apt repository*:

https://docs.docker.com/engine/install/ubuntu/

Verifique la instalación
```
sudo docker run hello-world
```

Realice ajustes de post instalación de Docker:

https://docs.docker.com/engine/install/linux-postinstall/

##12.2 Práctica con Docker

Se creará un contenedor que despliegue un sitio web creado con Flask.

**Paso 1:** Crear directorio temporal

Cree una carpeta en el equipo local con el nombre origen. Aquí se colocarán  todos los archivos que posteriormente se copiarían al contenedor.

**Paso 2:** Crear el archivo docker (Dockerfile)

Cree dentro de la carpeta origen un archivo llamado *Dockerfile*. Inserte las siguientes líneas:
- FROM toma como base una imagen, en este caso partimos de una imagen que ya tiene instalado Python versión 3:
```  
FROM python:3
```
- RUN para instalar Flask en el contenedor:
```
RUN pip install Flask
```
- WORKDIR agrega la ruta al directorio de trabajo, a la vez que se crea el directorio app:
```
WORKDIR /home/app
```
- COPY para pasar el archivo server.py dentro del contenedor. El origen está el archivo en el equipo local y el punto '.' Indica que el destino es en el directorio definido por WORKDIR:
```
COPY server.py .
```
- EXPOSE expone el puerto 8080 para su uso por el servidor web.
```
EXPOSE 8080
```
- CMD ejecuta el script de Python que inicia el servidor web:
```
CMD python3 /home/app/server.py
```

**Paso 3:** Construir y ejecutar el contenedor Docker

Construya la imagen del contenedor con el comando:
```
docker build -t sample .
```
La opción -t permite especificar el nombre de la imagen del contenedor (en este caso sample). El punto (.) indica que la imagen la genere en el directorio actual, donde está el Dockerfile.

Ejecute o inicie el contenedor:
```
docker run -t -d -p 8080:8080 --name samrun sample
```
La opción -p indica primero el puerto externo y luego el puerto interno.

**Paso 4:** Probar el funcionamiento del contenedor

Verifique con un buscador que el sitio web se desplegó adecuadamente.

**Paso 5:** Comandos adicionales

Para observar las imágenes:
```
docker images
```

Para observar los contenedores en ejecución (-a incluso los detenidos):
```
docker ps -a
```

Para detener el contenedor:
```
docker stop samrun
```
Para detener el contenedor:
```
docker start samrun
```
Para borrar una imagen:
```
docker rmi
```
Para eliminar permanentemente el contenedor, primero se debe detener y luego remover con:
```
docker rm
```
Para acceder al contenedor en ejecución:
```
docker exec -it samrun /bin/bash
```
donde samrub es el nombre del contenedor en ejecución y se selecciona el shell bash (/bin/bash).


#13. Jenkins

##13.1. Instalación

- Primero instale Java (Installation of Java) y luego instale Jenkins (Long Term Support release) desde:
https://www.jenkins.io/doc/book/installing/linux/
- Ajustes de post instalación de Jenkins:

  Permita a Jenkins ser del grupo de Docker:
```
sudo usermod -aG docker jenkins
```
- Reinicie el servicio Jenkins.
- Inicie Jenkins desde un buscador en su equipo. Jenkins se encuentra en el puerto 8080. Realice los ajustes solicitados para poner a punto Jenkins.

##13.2. Proyecto en Jenkins

**Paso 1** En Jenkins: Crear un Pipeline Job

En Jenkins:

Nuevo item > Ponle nombre (ej: PipeWeb) > selecciona Pipeline > OK.

**Paso 2** En Jenkins: Configurar

En la sección Pipeline:
 - En "Definition" elegir: Pipeline script
 - En el cuadro de texto pegar lo siguiente ajustando el URL del repositorio:

```
pipeline {
    agent any

    stages {
        stage('Limpiar Workspace') {
            steps {
                cleanWs()
            }
        }

        stage('Clonar repositorio') {
            steps {
                git url: 'https://github.com/usuario/repo.git', branch: 'main'
            }
        }

        stage('Ejecutar despliegue') {
            steps {
                sh '''
                    chmod +x script
                    ./script
                '''
            }
        }
    }

    post {
        success {
            echo 'Despliegue exitoso'
        }
        failure {
            echo 'Error durante el despliegue'
        }
    }
}
```

**Paso 3** En Jenkins: Configurar los triggers

En la sección Build Triggers:
- Marcar: GitHub hook trigger for GITScm polling

Es el que responde al webhook de GitHub.

**Paso 4** En Jenkins: Ajustar seguridad

GitHub no incluye un "crumb" (token anti-CSRF), por lo que Jenkins rechazará la solicitud.

Para deshabilitar la protección CSRF para /github-webhook.

En Jenkins:

Manage Jenkins → Security → CSRF Protection

Habilitar la opción: Enable proxy compatibility

**Paso 5** Probar el trabajo en Jenkins

Ejecutar el Job: Build Now

En el directorio de trabajo debe observarse el resultado:
```
/var/lib/jenkins/workspace/mi-job
```

**Paso 6** En Github: Crear el webhook

En el repositorio en GitHub > Settings > Webhooks > Add webhook

Configura así:
- Payload URL:
http://IP_JENKINS:8080/github-webhook/
- Content type: application/json
- Secret: puedes dejar vacío si no configuras la verificación en Jenkins.
- Which events would you like to trigger this webhook?: Just the push event

Clic en Add webhook

**Paso 7** En Github: Verificar Webhook

en GitHub > Settings > Webhooks

Verificar: Last delivery was successful

**Paso 8** En Github: Verificar

Hacer un commit en Github. Se debe activar el trabajo en Jenkins y ejecutar el Script.




